In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal

In [ ]:
import warnings

warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
pdd_path = 'data/dukcapil_penduduk_ntb.gpkg'
grid_path = 'data/grid_prov_52.geojson'
builtup_paths = '/content/MyDrive/MyDrive/NTB/Wilkerstat REGSOSEK/tag_regsosek.geojson'
dir_result = '/content/MyDrive/MyDrive/NTB'

In [ ]:
builtup = gpd.read_file(builtup_paths).to_crs(epsg='3857')
grid = gpd.read_file(grid_path).to_crs(epsg='3857')
pdd = gpd.read_file(pdd_path).to_crs(epsg='3857')

In [ ]:
pdd['kode_desa_spatial'] = pdd['kode_desa_spatial'].astype(str)

In [ ]:
points_in_regions = gpd.sjoin(builtup, pdd, how="inner", predicate="within")
points_in_regions_count = points_in_regions.groupby('index_right').size()

pdd['jml_bgn'] = pdd.index.map(points_in_regions_count).fillna(0).astype(int)

In [ ]:
grid_desa = gpd.overlay(pdd, grid, how="intersection")

In [ ]:
points_in_grids = gpd.sjoin(builtup, grid_desa, how="inner", predicate="within")
points_in_grids_count = points_in_grids.groupby('index_right').size()

grid_desa['jml_bgn_grid'] = grid_desa.index.map(points_in_grids_count).fillna(0).astype(int)

In [ ]:
grid_desa = grid_desa.merge(pdd[['jumlah_penduduk', 'kode_desa_spatial']], how='left')
grid_desa['pdd_grid_desa'] = grid_desa['jumlah_penduduk'] * grid_desa['jml_bgn_grid'] / grid_desa['jml_bgn']

In [ ]:
total_population_in_grids = grid_desa.groupby('id').agg({
    'pdd_grid_desa': 'sum'
}).reset_index()

In [ ]:
final_result = grid_desa[['id', 'geometry']].drop_duplicates(subset='id').set_index('id')

In [ ]:
temp = gpd.sjoin(grid, pdd, how="inner", predicate="intersects")
grid_ntb = temp[grid.columns]

In [ ]:
final_result = grid_ntb.merge(total_population_in_grids, on='id', how='left')

final_result['pdd_grid_desa'] = final_result['pdd_grid_desa'].fillna(0)

In [ ]:
final_result.to_file('final_grid.geojson')

In [ ]:
def centers(gdf, col):
  gdf = gdf[gdf[col]>=1500]

  # create spatial weights matrix
  W = libpysal.weights.Rook.from_dataframe(gdf)

  # get component labels
  components = W.component_labels

  combined_polygons = gdf.dissolve(by=components, aggfunc={col: 'sum'})
  result = combined_polygons[combined_polygons[col]>=50000]
  result = result.rename(columns={col: col})
  result['luas_cluster'] = result.geometry.area
  if not result.empty:
    result['kelas'] = 'urban_center'

  return result

In [ ]:
def clusters(gdf, col):
  gdf = gdf[gdf[col]>=300]

  # create spatial weights matrix
  W = libpysal.weights.Rook.from_dataframe(gdf)

  # get component labels
  components = W.component_labels

  combined_polygons = gdf.dissolve(by=components, aggfunc={col: 'sum'})
  result = combined_polygons[combined_polygons[col]>=5000]
  result = result.rename(columns={col: col})
  result['luas_cluster'] = result.geometry.area
  result['kelas'] = 'urban_clusters'

  return result

In [ ]:
def rural(gdf, col):
  gdf_1 = gdf[gdf[col]>=300]
  gdf_2 = gdf[gdf[col]<300]

  # create spatial weights matrix
  W1 = libpysal.weights.Rook.from_dataframe(gdf_1)
  W2 = libpysal.weights.Rook.from_dataframe(gdf_2)

  # get component labels
  components1 = W1.component_labels
  components2 = W2.component_labels

  combined_polygons1 = gdf_1.dissolve(by=components1, aggfunc={col: 'sum'})
  temp1 = combined_polygons1[combined_polygons1[col]<5000]
  temp1 = temp1.rename(columns={col: col})

  temp2 = gdf_2.dissolve(by=components2, aggfunc={col: 'sum'})
  temp2 = temp2.rename(columns={col: col})

  result = gpd.GeoDataFrame(pd.concat([temp2, temp1], ignore_index=True), crs=gdf_2.crs)
  Wr = libpysal.weights.Rook.from_dataframe(result)
  componentsr = Wr.component_labels

  result = result.dissolve(by=componentsr, aggfunc={col: 'sum'})
  result['luas_cluster'] = result.geometry.area
  result['kelas'] = 'rural_grid_cells'

  return result

In [ ]:
try:
  centers(final_result, 'pdd_grid_desa').to_file('centers.gpkg')
except:
  pass

clusters(final_result, 'pdd_grid_desa').to_file('clusters.gpkg')
rural(final_result, 'pdd_grid_desa').to_file('rural.gpkg')